In [1]:
#importing pandas module
import pandas as pd

In [2]:
# by default this library is not installed in the colab so first installing this library ......
# pls refer to the documentation of this library it will help a lot .....

!pip install ctrl4ai

     |████████████████████████████████| 6.8MB 11.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [56]:
from ctrl4ai import preprocessing
from ctrl4ai import automl
from ctrl4ai import datasets

In [6]:
#named it dataset

dataset = pd.read_csv('titanic.csv')

In [10]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
#seeing the no of columns and no of rows for analysis

dataset.head

<bound method NDFrame.head of      PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0              1         0       3  ...   7.2500   NaN         S
1              2         1       1  ...  71.2833   C85         C
2              3         1       3  ...   7.9250   NaN         S
3              4         1       1  ...  53.1000  C123         S
4              5         0       3  ...   8.0500   NaN         S
..           ...       ...     ...  ...      ...   ...       ...
886          887         0       2  ...  13.0000   NaN         S
887          888         1       1  ...  30.0000   B42         S
888          889         0       3  ...  23.4500   NaN         S
889          890         1       1  ...  30.0000  C148         C
890          891         0       3  ...   7.7500   NaN         Q

[891 rows x 12 columns]>

In [12]:
dataset.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [57]:
from ctrl4ai import helper
single_valued_column = [ i for i in dataset.columns
                         if helper.single_valued_col(dataset[i]) == True ]

In [15]:
# it gives the list of all single valued columns....i.e the columnsn which just have a single value in it...

single_valued_column

[]

In [42]:
# here the categorical threshold plays the very major role in choosing which columns can act as categorical variale and which column not....
# we neeed to be very catious about the threshold of categorical variable... and need to work more on this....
# think about how can we choose for the categorical threshold for the dataset... 
# we need to make the threshold value dynamic...

from ctrl4ai import helper
categorical_variable = [ i for i in dataset.columns
                         if helper.check_categorical_col(dataset[i],categorical_threshold=0.04) == True]


In [43]:
# this is the list of categorical variable ...this can be changed by changing the threshold of the categorical variable....

categorical_variable

['Survived', 'Pclass', 'Sex', 'Embarked']

In [44]:
# it basically converts the categorical variables to the dummy variable by droping one column in it so as to avoid dummy variable trap....

for e in range(len(categorical_variable)) :
    dummy = pd.get_dummies( dataset[categorical_variable[e]] , drop_first = True)
    if e == 0 :
        df = pd.concat([dataset , dummy] ,axis=1)

    df = pd.concat([df , dummy] ,axis=1)

In [45]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,1,1,2,3,male,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,0,1,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1,0,0,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,0,1,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1,0,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0,0,1,1,0,1


In [58]:
non_numeric_dataset = preprocessing.drop_non_numeric(dataset)

no_of_columns_dataset = dataset.columns

no_of_columns_non_numeric_dataset = non_numeric_dataset.columns


#items stored in list of temp3 is the elements which are there in no_of_columns_dataset but not in non_numeric_dataset...

temp3 = [item for item in no_of_columns_dataset if item not in non_numeric_dataset]

list_of_categorial_variables = categorical_variable

li4 = temp3

temp4 =  [item for item in temp3 if item in list_of_categorial_variables]

to_be_removed = [item for item in temp3 if item not in temp4]

new_data_set = dataset.drop(to_be_removed, axis=1)

Dropping non categorical/continuous column(s):Name,Sex,Ticket,Cabin,Embarked


In [47]:
new_data_set.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [48]:
# dropping the columns of the dataset which has null fields in it provided the threshold id 0.3
# here also we have to make the threshold value dyanamic  so that we don't have to rely on seeing the dataset...
# we need to make the variable values dynamic so as to make the code better....
# here in have fixed the threshold value to 0.3.......


new_data_set_1 = preprocessing.drop_null_fields(new_data_set, dropna_threshold=0.3)

In [51]:
# imputing th values in the null fields odf the dataframe 
# for continuos data we do mean and for categorical data we do mode over here....
# preprocessing functions help to decide with which way the nulll values must be filled so that 
# out dataset don't deviate much from actual dataset.......


dataset_with_no_null = preprocessing.impute_nulls(new_data_set, method='central_tendency')

In [52]:
# confirming again if the dataset after removing and computing the null values is still left with any nulll values or not....
# it comes false which means it has no null values in the entiree dataset......

dataset_with_no_null.isnull().values.any()

False

In [54]:
data_set_with_dummy = pd.get_dummies(dataset_with_no_null, columns=categorical_variable, drop_first=True)

In [55]:
data_set_with_dummy.head()

,PassengerId,Age,SibSp,Parch,Fare,Survived_1,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,1,22.0,1,0,7.2500,0,0,1,1,0,1
1,2,38.0,1,0,71.2833,1,0,0,0,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,1
3,4,35.0,1,0,53.1000,1,0,0,0,0,1
4,5,35.0,0,0,8.0500,0,0,1,1,0,1
